In [27]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
df_reviews = pd.read_csv(r"C:\Users\nikit\Documents\Graduation internship\NLP\Datasets\dutch_book_reviews.csv")
df_reviews.head(15)

,review,sentiment,IsPositive
0,'Can love ever be stronger than fate?' dat sta...,positive,1
1,Er wordt in het verhaal gebruik gemaakt van gl...,positive,1
2,Verder vond ik het een origineel verhaal met e...,positive,1
3,Ik ben gaan meeleven met Ella en vind het erg ...,positive,1
4,Lees ook het interview met N.J. Simmons,positive,1
5,Wat een geweldig boek is dit weer van Loes! Ik...,positive,1
6,Bij het zien van Arthur Nebe op de cover verwa...,positive,1
7,Op een boeiende en vlotte manier vertelt Kevin...,positive,1
8,Het is niet alleen het verhaal van Arthur Nebe...,positive,1
9,Lees verder op https://surfingann.blogspot.nl/...,positive,1


In [30]:
df_reviews.IsPositive.value_counts()

1    58776
0    44370
Name: IsPositive, dtype: int64

In [36]:
nlp = spacy.load("nl_core_news_sm")
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return " ".join(filtered_tokens)

In [38]:
df_reviews['preprocessed_txt'] = df_reviews.review.apply(preprocess)
df_reviews.head()

,review,sentiment,IsPositive,preprocessed_txt
0,'Can love ever be stronger than fate?' dat sta...,positive,1,Can love ever be stronger than fate staan cove...
1,Er wordt in het verhaal gebruik gemaakt van gl...,positive,1,verhaal gebruik maken glimpen verleden hoofdst...
2,Verder vond ik het een origineel verhaal met e...,positive,1,vinden origineel verhaal interessant kijk leve...
3,Ik ben gaan meeleven met Ella en vind het erg ...,positive,1,gaan meeleav Ella vinden erg 2018 wachten wach...
4,Lees ook het interview met N.J. Simmons,positive,1,Lees interview N.J. Simmons


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_reviews.preprocessed_txt, df_reviews.IsPositive, test_size= 0.2)

In [41]:
X_train.shape

(82516,)

In [42]:
X_test.shape

(20630,)

In [43]:
X_train[:4]

C:\Users\nikit\AppData\Local\Temp\ipykernel_20040\2345937334.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train[:4]


96378    hoofdpersonage sympathie oproepen langdradig b...
63245                                  dood langzaam leeft
63634    vrouw mooi lief mysterieus allemaal sociaal na...
7636     titel geven boek ontwikkeling zitten persoon I...
Name: preprocessed_txt, dtype: object

In [44]:
y_train[:4]

C:\Users\nikit\AppData\Local\Temp\ipykernel_20040\291300964.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train[:4]


96378    0
63245    1
63634    0
7636     0
Name: IsPositive, dtype: int64

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(ngram_range= (1, 3))
X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<82516x2947629 sparse matrix of type '<class 'numpy.int64'>'
	with 5434377 stored elements in Compressed Sparse Row format>

In [10]:
X_train_cv.toarray()[:2][0]

MemoryError: Unable to allocate 2.36 TiB for an array with shape (82516, 3925870) and data type int64

In [46]:
v.vocabulary_

{'hoofdpersonage': 1024764,
 'sympathie': 2288029,
 'oproepen': 1810513,
 'langdradig': 1322758,
 'beschrijving': 241359,
 'personage': 1879940,
 'echt': 615998,
 'leven': 1367175,
 'komen': 1252810,
 'motief': 1617992,
 'compleet': 440320,
 'onderbelicht': 1726670,
 'blijven': 302764,
 'kunnen': 1301006,
 'olssons': 1711326,
 'thrillerdebuut': 2349627,
 'samenvatten': 2072922,
 'hoofdpersonage sympathie': 1026213,
 'sympathie oproepen': 2288253,
 'oproepen langdradig': 1810841,
 'langdradig beschrijving': 1322828,
 'beschrijving personage': 242573,
 'personage echt': 1881548,
 'echt leven': 620447,
 'leven komen': 1372272,
 'komen motief': 1262315,
 'motief compleet': 1618076,
 'compleet onderbelicht': 440760,
 'onderbelicht blijven': 1726679,
 'blijven kunnen': 305767,
 'kunnen olssons': 1301619,
 'olssons thrillerdebuut': 1711327,
 'thrillerdebuut samenvatten': 2349755,
 'hoofdpersonage sympathie oproepen': 1026214,
 'sympathie oproepen langdradig': 2288257,
 'oproepen langdradig be

In [47]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [48]:
X_test_cv = v.transform(X_test)

In [49]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test_cv)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.61      0.69      8936
           1       0.75      0.89      0.81     11694

    accuracy                           0.77     20630
   macro avg       0.77      0.75      0.75     20630
weighted avg       0.77      0.77      0.76     20630



In [50]:
rev = ['''De kwaliteit die ik van Vogel's gewend ben. Mooie vormgeving. De handleiding is duidelijk en overzichtelijk. De onderdelen zijn heel handig verpakt zodat je precies weet wat je nodig hebt.''','''Hoewel bedoeld voor de professional is het systeem ook door een handige doe-het-zelver te monteren. In mijn geval bovenaan de wand boven de schuifdeuren. Dus niet alleen geschikt voor plafondmontage en een schuin plafond maafr ook voor aan de wand. Wel heel zorgvuldig tewerk gaan want je wilt niet dat tv omlaag komt.
Alleen de twee zwarte afdekkappen vind ik niet zo mooi omdat ze nog een aantal bouten in het zicht laten.''','''Zou zogezegd passen op epson projectors maar dit is niet waar. (toch allesinds voor Epson TW7100) je kan de beugel niet vastmaken aan de projector.''','''Handig en gemakkelijk te plaatsen met de ingebouwde waterpas en de stap voor stap handleiding.''','''Fijne steun, echter deze een beetje moeten aanpassen omdat deze toch niet helemaal paste op onze oude TV, terwijl ik wel het juiste nummer had gevonden. De voedingskabel kan er nu niet uit.''','']
rev_count = v.transform(rev)
model.predict(rev_count)

array([1, 1, 1, 1, 1, 1], dtype=int64)

In [57]:
df_vogel = pd.read_csv(r"C:\Users\nikit\Documents\Graduation internship\NLP\Datasets\vogel.csv")
df_vogel = df_vogel.drop(['web-scraper-order', 'web-scraper-start-url', 'products', 'products-href','rating ', 'next','next-href','review1'], axis = 1)
df_vogel = df_vogel.dropna()
df_vogel['preprocessed_txt'] = df_vogel.review.apply(preprocess)
df_vogel.head()

,review,preprocessed_txt
1,"Prima artikel, echter systeem niet feilloos. W...",Prima artikel systeem feilloos wallplaar volle...
2,Het systeem is makkelijk op te hangen. Alle be...,systeem makkelijk afhangen benodighed zitten e...
3,"Perfecte kwaliteit, simpel met het monteren va...",perfect kwaliteit simpel monteren beugel tv mu...
4,"Ik heb dit vorige week gekocht, met het gedach...",vorig week kopen gedachte 65 inch tv afhangen ...
5,Een wall mount die supergemakkelijk te install...,wall mount supergemakkelijk installeren max 10...


In [59]:
rev_count = v.transform(df_vogel.preprocessed_txt)
model.predict(rev_count)

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1], dtype=int64)

In [60]:
a = model.predict(rev_count)
a = pd.DataFrame(a)
print(a.value_counts())

1    36
0     9
dtype: int64
